## Import and load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!pip install bert-sklearn/.

In [ ]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [ ]:
ROOT_PATH = '/content/drive/My Drive/Colab Notebooks (1)/sentiment_analysis/'

In [5]:
train = pd.read_csv(ROOT_PATH + 'data/train_add_gen.csv', encoding='utf-16')
print(train.shape)
train.head()

(10187, 2)


,content,ground
0,Đôi khi màn hình chạy chậm...Vào mạng nhanh nó...,0
1,Dùng tốt nhưng thiếu 4g và bộ nhớ hơi kém và m...,0
2,Vào các ứng dụng hầu như rất chậm. Chơi game n...,0
3,Giảm gần 1 nữa lúc mua. F9 ra còn rẻ hơn f7 lú...,0
4,"Đã mua và sử dụng được 2 tháng, rất tuyệt. Xài...",1


In [ ]:
from collections import Counter
Counter(train.ground)

Counter({0: 5084, 1: 5103})

In [10]:
test = pd.read_csv(ROOT_PATH + 'data/test.csv', encoding='utf-16')
print(test.shape)
test.head()

(1552, 2)


,content,ground
0,Sản phẩm hay bị đơ! Dùng pin 4g hao nhanh. Hỗ ...,0
1,"Đẹp nhất, cấu hình cao nhất, pin trâu nhất tro...",1
2,"Trên cả tuyệt vời 😍 pin trâu, mượt, sang trọng...",1
3,Sản phẩm tầm trung Xài tạm ổn. Nói chung sản p...,0
4,oppo neo 7 thật quá đã. Tính năng cũng như HĐH...,1


## Process data

In [ ]:
def text_normalize(df, tokenize=False):
    # Convert text to lowercase
    df['content'] = df['content'].str.lower()
    # Remove numbers and words with numbers
    df['content'] = df['content'].str.replace('\w*\d\w*', ' ')
    # Remove punctuation
    df['content'] = df['content'].str.replace('[^\w\s]', ' ')
    # Remove whitespaces
    df['content'] =  df['content'].str.split().apply(lambda x : ' '.join(word for word in x))
    # Tokenize
    if tokenize:
        df['content'] = df['content'].apply(lambda x : ViTokenizer.tokenize(x))

In [12]:
text_normalize(train)
text_normalize(test)
test.tail()

,content,ground
1547,máy mình dùng một thời gian ngắn nhưng chậm sử...,0
1548,cho đến hôm nay máy vẫn bị loạn cảm ứng cập nh...,0
1549,đẹp chuẩn sài rất êm tôi rất hài lòng về dòng ...,1
1550,mặc dù mua máy cũ nhưng trông như máy mới mọi ...,1
1551,máy chạy tốt mình hài lòng mua từ tháng năm tớ...,0


In [ ]:
X_train = train['content']
y_train = train['ground']

X_test = test['content']
y_test = test['ground']

## BERT model

In [ ]:
model = BertClassifier(max_seq_length=128,
                       train_batch_size=32,
                       epochs=5,
                       bert_model='bert-base-multilingual-cased')
model

Building sklearn text classifier...


BertClassifier(bert_config_json=None, bert_model='bert-base-multilingual-cased',
               bert_vocab=None, do_lower_case=None, epochs=5, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)

## Train

In [ ]:
%%time
history = model.fit(X_train, y_train)

Loading bert-base-multilingual-cased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 9169, validation data size: 1018




Epoch 1, Train loss: 0.3272, Val loss: 0.2379, Val accy: 89.78%




Epoch 2, Train loss: 0.1736, Val loss: 0.2120, Val accy: 91.55%




Epoch 3, Train loss: 0.0955, Val loss: 0.2797, Val accy: 91.36%




Epoch 4, Train loss: 0.0536, Val loss: 0.2666, Val accy: 93.03%




Epoch 5, Train loss: 0.0286, Val loss: 0.3034, Val accy: 93.03%

CPU times: user 9min 14s, sys: 3min 55s, total: 13min 10s
Wall time: 13min 29s


## Test

In [ ]:
from tqdm import tqdm
# score model
accy = model.score(X_test, y_test)

# make class probability predictions
y_prob = model.predict_proba(X_test)
print("class prob estimates:\n", y_prob)

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, y_test) * 100))

target_names = ['negative', 'positive']
print(classification_report(y_test, y_pred, target_names=target_names))


Loss: 0.4273, Accuracy: 89.43%


class prob estimates:
 [[9.7782636e-01 2.2173658e-02]
 [8.9391734e-04 9.9910611e-01]
 [1.1498650e-03 9.9885011e-01]
 ...
 [9.5372571e-04 9.9904627e-01]
 [3.0348129e-03 9.9696523e-01]
 [3.2398552e-03 9.9676019e-01]]


Accuracy: 89.43%
              precision    recall  f1-score   support

    negative       0.91      0.87      0.89       758
    positive       0.88      0.92      0.90       794

    accuracy                           0.89      1552
   macro avg       0.90      0.89      0.89      1552
weighted avg       0.89      0.89      0.89      1552



In [ ]:
y_pred[:10]

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1])

In [ ]:
y_test[:10].values

array([0, 1, 1, 0, 1, 0, 1, 0, 0, 1])

In [ ]:
print('Ground\tPred\tText')
for i in range(10):
    print(str(y_test[i]) + '\t' + str(y_pred[i]) + '\t' + X_test[i])

Ground	Pred	Text
0	0	sản phẩm hay bị đơ dùng pin hao nhanh hỗ trợ sạc nhanh ko thật sự hiệu quả chụp ảnh chỉ dừng ở mức chập nhận đc ko lung linh như hãng khác cùng tầm giá
1	1	đẹp nhất cấu hình cao nhất pin trâu nhất trong tầm giá rất hợp với một kiến trúc sư
1	1	trên cả tuyệt vời pin trâu mượt sang trọng không rời em nó nửa bước cây bút dễ dùng
0	1	sản phẩm tầm trung xài tạm ổn nói chung sản phâm bình thường không có ưu điểm nổi bât
1	1	oppo neo thật quá đã tính năng cũng như hđh phù hợp với giới trẻ hiện nay nhưng giá cả nê giảm lại một chút nữa thì bán chạy hơn nhiều
0	0	lúc đầu mình sử dụng cũng hay bị lõi nhưng khi cập nhật phần mềm vá lỗi thì ok với mức giá tôi không thích ghét cái sóng mạng rất yếu nhất là cái phần mềm nâng cấp nó làm mất đi chức năng tiện lợi hữu ít lúc đầu như chỉnh độ sáng đen pin chỉnh máy ảnh v v lúc trước nói đến samsung là nói đến chất lượng tôi không hiểu sao samsung càng ngày càng tệ bó tay luôn
1	1	mới mua hồi sáng về chơi tới giờ pin còn hơn đúng trâ

# Save model

In [ ]:
savefile = ROOT_PATH + 'resource/BERT_0102.bin'

In [ ]:
# save model to disk
model.save(savefile)

In [14]:
# load model
BERT = load_model(savefile)

BERT.score(X_test, y_test)

Loading model from /content/drive/My Drive/Colab Notebooks (1)/sentiment_analysis/resource/BERT_0102.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...




Loss: 0.4273, Accuracy: 89.43%


89.43298969072166